In [1]:
%load_ext autoreload
%autoreload 1
%aimport ecg_get_data
%aimport Models
%aimport train_test_validat
%aimport self_attention
%aimport ECGplot
%aimport Net
import Models 
import Net
from train_test_validat import *
from self_attention import *
import  ecg_get_data 
import matplotlib.pyplot as plt
import ecg_plot

import torch
import torch.utils.data as Data
import random

import time
import os
import gc

random_seed = 2
torch.manual_seed(random_seed)    # reproducible
torch.cuda.manual_seed_all(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)

time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime()) 

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_npy_path =  './data/train/' #路径
test_npy_path =  './data/test/' 
shadow_npy_folder = './data/addition/'
xml_path = './xml/xml/'
#lable_path = './label.npy'

model_path = './model/'+time_str
log_path = './log/'+  time_str


EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)
print(torch.cuda.get_device_name(0))# 返回gpu名字

cuda:0
NVIDIA GeForce RTX 3090 Ti


In [3]:
test_Dataset = ecg_get_data.ECG_Dataset(test_npy_path,EcgChannles_num,EcgLength_num,xml_folder=xml_path)
x_Dataset = ecg_get_data.ECG_Dataset(train_npy_path,EcgChannles_num,EcgLength_num,shadow_npy_folder=shadow_npy_folder)

npys:{%d} 100
npys:{%d} 3866
shadow_npys:{%d} 4461


In [4]:
ECG,label = test_Dataset.__getitem__(1)
#inf,path = train_Dataset.get_basic_inf(55)
#ecg_plot.plot(ECG*3500/1000, sample_rate = 500, title = "test",row_height= 10,show_grid=True,show_separate_line=True)
label
#ecg_plot.save_as_png(inf[1],'/workspace/data/OneDrive - mail.hfut.edu.cn/ECG/Interpretable_HTN//PNG_ECG/',dpi = 100)

tensor(1)

In [5]:
BATCH_SIZE = 64

FOLDS = 1
EPOCHS = 5000  
PATIENCE = 15
LR = 0.005

In [6]:
from torch.utils.tensorboard import SummaryWriter   
os.makedirs(model_path, exist_ok=True)
writer = SummaryWriter(log_path)
from torchsummary import summary

In [9]:
torch.cuda.empty_cache()# 清空显卡cuda
#NET = [Models.channels_split_ATT_CNN_(mark=True) for i in range(FOLDS)]
NET = [
    Net.channels_branch_CNN(True)
    ]
# NET = [ 
#         Models.channels_split_ATT_CNN_linear_avgpool_for_grad(mark=True,extract_dim=16,hdim=32)
#       ]
# NET = [Models.resnet18(12,64,2)]
#NET = [Models.channels_split_ATT_CNN_(mark=True) for i in range(FOLDS)]
#NET =[Models.Informer(1)]

In [10]:
from torch.optim.lr_scheduler import CosineAnnealingLR
torch.cuda.empty_cache()# 清空显卡cuda
for fold in range(FOLDS):
    train_size= int(0.8*len(x_Dataset))
    validate_size=(len(x_Dataset)) - train_size
    train_Dataset,valid_Dataset=torch.utils.data.random_split(x_Dataset,[train_size,validate_size])    # type: ignore
    early_stopping = EarlyStopping(PATIENCE, verbose=True, model_path=model_path, delta=0)
    #train_dataset,valid_dataset = get_k_fold_dataset(fold=int(fold+1),x = train_x,y=train_y,k=FOLDS,random_seed = random_seed)
    train_dataloader = Data.DataLoader(dataset=train_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=5,pin_memory=True)
    valid_dataloader = Data.DataLoader(dataset=valid_Dataset, batch_size=BATCH_SIZE, shuffle=True,num_workers=5,pin_memory=True)
    NET[fold].to(DEVICE)
    optimizer  = torch.optim.Adadelta(NET[fold].parameters(), lr=LR,weight_decay=1e-2)  
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 7, gamma=0.5)
    criterion = torch.nn.CrossEntropyLoss()   

    #等间隔调整学习率
    #scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,T_max = 24)
    best_loss = 3
    for epoch in range(1,EPOCHS):
        time_all=0
        start_time = time.time()
        train_loss,train_acc = train_model(train_dataloader, NET[fold], criterion, optimizer,DEVICE) # 训练模型

        time_all = time.time()-start_time
        y_true,y_pred,validate_loss,validate_acc = eval_model(valid_dataloader,criterion,NET[fold],DEVICE) # 测试模型
        F1_score =f1_score(y_true, y_pred, average='macro')#F1分数

        writer.add_scalars(main_tag=str(fold)+'_Loss',tag_scalar_dict={'train': train_loss,'validate': validate_loss},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_Accuracy',tag_scalar_dict={'train': train_acc,'validate': validate_acc},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_LearningRate',tag_scalar_dict={'LR': optimizer.state_dict()['param_groups'][0]['lr']},global_step=epoch)
        writer.add_scalars(main_tag=str(fold)+'_F1_score',tag_scalar_dict={'_F1_score': F1_score},global_step=epoch)

        print('- Epoch: %d - Train_loss: %.5f - Train_acc: %.5f - Val_loss: %.5f - Val_acc: %5f - T_Time: %.5f' %(epoch,train_loss,train_acc,validate_loss,validate_acc,time_all))
        print('F1 score: %f' %(F1_score))
        print('当前学习率：%.8f' %optimizer.state_dict()['param_groups'][0]['lr'])

        if validate_loss < best_loss:
            best_loss = validate_loss
            print('Find better model in Epoch {0}, saving model.'.format(epoch))
            # torch.save(NET[fold],  model_path+'/all_best_model_' + str(fold) + '.pt')  # 保存最优模型
            torch.save(NET[fold].state_dict(), model_path+'/parameter_best_model_' + str(fold) + '.pt')
        else:
            scheduler.step() # 学习率迭代
        #是否满足早停法条件
        if(early_stopping(F1_score,NET[fold],fold)):
            print("Early stopping")
            break

    print('Fold %d Training Finished' %(fold+1))
    torch.cuda.empty_cache()# 清空显卡cuda
print('Training Finished')

- Epoch: 1 - Train_loss: 0.68618 - Train_acc: 0.55886 - Val_loss: 0.68133 - Val_acc: 0.563301 - T_Time: 57.66569
F1 score: 0.487528
当前学习率：0.00500000
Find better model in Epoch 1, saving model.
- Epoch: 2 - Train_loss: 0.67602 - Train_acc: 0.58297 - Val_loss: 0.67162 - Val_acc: 0.565705 - T_Time: 56.76784
F1 score: 0.500958
当前学习率：0.00500000
Find better model in Epoch 2, saving model.
- Epoch: 3 - Train_loss: 0.66839 - Train_acc: 0.59994 - Val_loss: 0.66955 - Val_acc: 0.549679 - T_Time: 56.21472
F1 score: 0.514418
当前学习率：0.00500000
Find better model in Epoch 3, saving model.
- Epoch: 4 - Train_loss: 0.66384 - Train_acc: 0.61716 - Val_loss: 0.65888 - Val_acc: 0.576522 - T_Time: 55.74019
F1 score: 0.513609
当前学习率：0.00500000
Find better model in Epoch 4, saving model.
EarlyStopping counter: 1 out of 15

- Epoch: 5 - Train_loss: 0.65534 - Train_acc: 0.64311 - Val_loss: 0.66189 - Val_acc: 0.555689 - T_Time: 56.52486
F1 score: 0.519360
当前学习率：0.00500000
- Epoch: 6 - Train_loss: 0.65073 - Train_ac

In [ ]:
class exectal_attention(nn.Module):
    def __init__(self,in_channels,out_channels,menmory_dim):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.menmory_dim = menmory_dim
        self.M_k = nn.Linear(self.out_channels, self.menmory_dim,bias=False)
        self.M_v = nn.Linear(self.menmory_dim, self.out_channels,bias=False)
        self.query = nn.Linear(self.in_channels, self.out_channels,bias=False)
        self.softmax = nn.Softmax(dim=1)
    def l1_norm(self,aij,dim=2):
        aij_norm= aij/torch.sum(aij,dim=dim)  # type: ignore
        return aij_norm
    def forward(self, x):
        x = self.query(x)
        attn = self.M_k(x)
        attn = self.softmax(attn)
        attn = self.l1_norm(attn,dim=2)
        out = self.M_v(x)
        return out,attn
